In [1]:
! pip install --user sentence-transformers 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f3207e2d1fce4e31783d68503a12191dbdf6930d7ddbdb1a7369fb752c55ed25
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd
from json import loads
from ast import literal_eval
df = pd.read_csv(filepath_or_buffer='/kaggle/input/text-and-meta-data-analysis/train.csv', )
df['perplexity_score'] = df['meta'].apply(func=lambda x: literal_eval(x)['perplexity_score'])
df['pile_set_name'] = df['meta'].apply(func=lambda x: literal_eval(x)['pile_set_name'])
df = df.drop(columns=['meta'])
df = df.dropna(subset=['text'])
df.head()

,text,perplexity_score,pile_set_name
0,"It is done, and submitted. You can play “Survi...",327.0,Pile-CC
1,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<seg...",977.7,Github
2,Article content\n\nHuman behavior has a tremen...,609.4,OpenWebText2
3,Topic: reinvent midnight madness\n\nAmazon ann...,262.3,Pile-CC
4,About Grand Slam Fishing Charters\n\nAs a fami...,858.9,Pile-CC


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99996 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   text              99996 non-null  object 
 1   perplexity_score  99996 non-null  float64
 2   pile_set_name     99996 non-null  object 
dtypes: float64(1), object(2)
memory usage: 3.1+ MB


In [4]:
# Let's clean up the text
df['clean'] = df['text'].str.replace('\n', ' ') # remove embedded newlines
clean_df = df.drop(columns=['text', 'perplexity_score', 'pile_set_name', ]).rename(columns={'clean': 'text'})
clean_df.sample(n=5, random_state=2023)

,text
31934,Flickr Open / Getty Images Old paint peeling f...
18263,Privacy Policy Samparknow is always there to ...
12731,Limited CapacityfullAdding this to your schedu...
76803,"Introduction {#S0001} ============ Recently, ..."
62813,Q: When I run rails server it is showing erro...


In [5]:
# https://www.kaggle.com/code/ianalyticsgeek/keywords-extraction-using-bert
from arrow import now
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# each of these is somewhat arbitary
N = 5000
MAX_FEATURES = 300
TOP_N = N // 10

# while we are testing let's take a small sample
sample_df = clean_df.sample(n=N, random_state=2023)
fit_data = sample_df['text'].values.tolist()

NGRAM_RANGE = [(1, 1), (2, 2)][0] # todo figure out the (2, 2) case
model_start = now()
count = TfidfVectorizer(ngram_range=NGRAM_RANGE, stop_words='english', max_features=MAX_FEATURES).fit(fit_data)
features = count.get_feature_names_out()
print('{}: got features'.format(now()))
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embedding = model.encode(fit_data)
print('{}: got embeddings'.format(now()))
feature_embeddings = model.encode(features)
print('{}: got feature embeddings'.format(now()))
print('model time: {}'.format(now() - model_start))

# use the cosine distance to measure how far the features are from the documents
distances = cosine_similarity(embedding, feature_embeddings)
# pick out the top few phrases to be prevalent keywords
phrases = [features[index] for index in distances.argsort()[0][-TOP_N:]]
# filter out numeric values
phrases = [item for item in phrases if not item.isnumeric()]
print(phrases)

2023-11-30T02:13:02.671901+00:00: got features


Batches:   0%|          | 0/157 [00:00<?, ?it/s]

2023-11-30T02:19:54.173069+00:00: got embeddings


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2023-11-30T02:19:55.239922+00:00: got feature embeddings
model time: 0:06:56.360489
['just', 'like', 'really', 'sure', 'll', 'according', 'got', 'right', 'let', 'way', 've', 'say', 'said', 'given', 'states', 'term', 'course', 'means', 'did', 'told', 'real', 'form', 'version', 'know', 'does', 'including', 'create', 'units', 'use', 'known', 'called', 'having', 'add', 'think', 'come', 'took', 'thing', 'air', 'set', 'place', 'world', 'true', 'area', 'using', 'provide', 'suppose', 'expression', 'var', 'et', 'available', 'head', 'fact', 'case', 'look', 'terms', 'time', 'value', 'national', 'non', 'rm', 'used', 'didn', 'program', 'al', 'present', 'went', 'based', 'open', 'make', 'point', 'example', 'great', 'begin', 'include', 'center', 'process', 'start', 'able', 'state', 'want', 'view', 'return', 'came', 'single', 'fig', 'going', 'direction', 'united', 'read', 'following', 'possible', 'similar', 'data', 'node', 'values', 'effect', 'period', 'order', 'ref', 'size', 'path', 'shown', 'informat

In [6]:
# now we can project the feature vectors into 2-space to visualize 
from plotly.express import scatter
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=2023, verbose=1, n_iter=10000)
tsne_df = pd.DataFrame(data=tsne.fit_transform(X=feature_embeddings), columns=['t0', 't1'])
tsne_df['word'] = features
scatter(data_frame=tsne_df[tsne_df['word'].isin(phrases)], x='t0', y='t1', text='word', height=900, ).update_traces(marker={'size': 1})

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 300 samples in 0.000s...
[t-SNE] Computed neighbors for 300 samples in 0.051s...
[t-SNE] Computed conditional probabilities for sample 300 / 300
[t-SNE] Mean sigma: 3.290262
[t-SNE] KL divergence after 250 iterations with early exaggeration: 58.017811
[t-SNE] KL divergence after 3250 iterations: 0.931782
